In [20]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import cv2

import sys, os, itertools, pickle
from pprint import pprint

from util import *
from config import *
from putil import *
%load_ext autoreload
%aimport util
%aimport putil
%aimport config
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def next_k(it, k):
    return [next(it) for _ in range(k)][-1]

def show_img(imgs, figsize=(12, 9)):
    if type(imgs) != list:
        imgs = [imgs]
    plt.figure(figsize=figsize)
    alpha = 1
    alpha_dec = 1 / len(imgs)
    for img in imgs:
        plt.imshow(img, alpha=alpha)
        alpha -= alpha_dec
    plt.axis('off')
    plt.show()
    
def to_homo(x):
    for axis, size in enumerate(x.shape):
        if size == 2:
            break
    else:
        raise RuntimeError()
    ones_shape = list(x.shape)
    ones_shape[axis] = 1
    return np.concatenate((x, np.ones(ones_shape)), axis=axis)

def mult_homo(M, x):
    n, d = x.shape
    if d == 2:
        x = to_homo(x)
    output = np.dot(x, M.T)
    return output[:, :2] / output[:, 2:3]

In [3]:
cached_frames_path = Root + 'cached_frames.p'
if os.path.exists(cached_frames_path):
    frames = load_pickle(cached_frames_path)
else:
    frames = {}
    for serie in 'pool_room', 'cory_breezeway', 'soda_front':
        calibrated_dir = Data + serie + '/calibrated/'
        videos = [read_video(get_data_path(calibrated_dir, i, name_is_dir=True)) for i in Cam_ids]
        k = 200
        for cam, video in zip(Cam_ids, videos):
            for i, frame in enumerate(video):
                if i % k == 0:
                    frames.setdefault((serie, i), {})[cam] = np.array(frame)
        for video in videos:
            video.close()
    save_pickle(frames, cached_frames_path)

In [24]:
cameras = { i : np.load(os.path.join(Calibrations, str(i), 'new_camera_matrix.npy')) for i in Cam_ids }

In [5]:
# { (camera1, camera2) : ((y_min, y_max, x_min, x_max)1, (y_min, y_max, x_min, x_max)2) }
masks = {}
masks[(1, 2)] = ((0, Video_height, 2000, Video_width), (0, 600, 300, 2000))
masks[(1, 3)] = ((0, Video_height, 0, 700), (1500, Video_height, 0, 2100))
masks[(1, 4)] = ((0, 600, 300, 2000), (0, Video_height, 2150, Video_width))
masks[(1, 5)] = ((1400, Video_height, 600, 2100), (0, Video_height, 2000, Video_width))
masks[(2, 4)] = ((0, Video_height, 2100, Video_width), (0, 400, 650, 2100))
masks[(2, 5)] = ((0, Video_height, 0, 650), (1400, Video_height, 600, 2100))
masks[(2, 6)] = ((1600, Video_height, 800, 2350), (0, Video_height, 2150, Video_width))
masks[(3, 4)] = ((0, Video_height, 2000, Video_width), (1500, Video_height, 500, 2200))
masks[(3, 5)] = ((0, Video_height, 0, 500), (0, 400, 650, 2100))
masks[(3, 6)] = ((0, 500, 500, 2200), (0, Video_height, 0, 600))
masks[(4, 6)] = ((0, Video_height, 0, 700), (1400, Video_height, 300, 2100))
masks[(5, 6)] = ((0, Video_height, 0, 700), (0, 400, 600, 2250))

def get_mask(y_min, y_max, x_min, x_max):
    mask = np.zeros((Video_height, Video_width), dtype=np.uint8)
    mask[y_min : y_max, x_min : x_max] = 1
    return mask

def in_mask(kp, y_min, y_max, x_min, x_max):
    x, y = kp.pt
    return (y_min <= y < y_max) and (x_min <= x < x_max)

full_masks = {} # { camera : overlap of all masks }
for c in Cam_ids:
    mask = None
    for (c1, c2), (cors1, cors2) in masks.items():
        if c1 == c:
            cors = cors1
        elif c2 == c:
            cors = cors2
        else:
            continue
        new_mask = get_mask(*cors)
        if mask is None:
            mask = new_mask
        else:
            mask |= new_mask
    full_masks[c] = mask

In [ ]:
for (c1, c2), (corners1, corners2) in sorted(masks.items()):
    show_img([frames['pool_room', 0][c1], get_mask(*corners1)], figsize=(8, 6))
    show_img([frames['pool_room', 0][c2], get_mask(*corners2)], figsize=(8, 6))

In [17]:
def find_match_indices(fs, keypoints, masks, c1, c2, debug=False):
    cors1, cors2 = masks[c1, c2]
    def get_keypoint_indices(c, cors):
        kps, des = keypoints[c] # all keypoints for a given camera
        # unique ids of points in the mask
        kp_ids = [i for i, p in enumerate(kps) if in_mask(p, *cors)]
        # unique points in the mask
        kps_masked = [kps[i] for i in kp_ids]
        des_masked = np.array([des[i] for i in kp_ids])
        return kps_masked, des_masked, kp_ids
    kp1, des1, kp_ids1 = get_keypoint_indices(c1, cors1)
    kp2, des2, kp_ids2 = get_keypoint_indices(c2, cors2)
#     show_img([fs[c1], get_mask(*cors1)])
#     show_img([fs[c2], get_mask(*cors2)])

    k = 2
    if len(des1) < k or len(des2) < k:
        return None
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    good = [m for m, n in matches if m.distance < 0.6 * n.distance]
    match_indices = [(kp_ids1[m.queryIdx], kp_ids2[m.trainIdx]) for m in good]

    if debug:
        if len(good) >= 8:
            pts1, pts2 = map(np.array, zip(*[(kp1[m.queryIdx].pt, kp2[m.trainIdx].pt) for m in good]))

            H, mask = cv2.findHomography(pts1, pts2, method=cv2.RANSAC, ransacReprojThreshold=1)
            mask = mask.ravel().tolist()
            side_by_side = cv2.drawMatches(
                np.array(fs[c1]), kp1,
                np.array(fs[c2]), kp2,
                good,
                None,
                matchesMask=mask,
                matchColor=(0, 255, 0),
                singlePointColor=(255, 0, 0),
                flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT
            )
            show_img(side_by_side, figsize=(24, 9))
        else:
            print('Not enough good points: only %s' % len(good))
    if len(match_indices) < 8:
        return None
    return match_indices

def compute_keypoints(frames, masks, full_masks):
    keypoints = {} # { camera : (keypoints, descriptors)}
    for im, mask in full_masks.items():
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(frames[im], mask)
        keypoints[im] = (kp, des)
    
    correspondences = {}
    for c1, c2 in masks.keys():
        match_indices = find_match_indices(fs, keypoints, masks, c1, c2)
        if match_indices:
            correspondences[c1, c2] = match_indices
#         print('%s correspondences between images %s and %s' % (len(match_indices), c1, c2))
    return keypoints, correspondences

In [ ]:
import random
for label, fs in random.sample(frames.items(), 5):
    # all keypoints within a frame
    keypoints = {} # { camera : (keypoints, descriptors) }
    for im, mask in full_masks.items():
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(fs[im], mask)
        keypoints[im] = (kp, des)
        
    for c1, c2 in random.sample(masks.keys(), 3):
        print(label, c1, c2)
        find_match_indices(fs, keypoints, masks, c1, c2, debug=True)

In [ ]:
index_maps = { c : {} for c in Cam_ids }
merged_keypoints = { c : [] for c in Cam_ids }
merged_correspondences = {}
for label, fs in frames.items():
    print(label)
    keypoints, correspondences = compute_keypoints(fs, masks, full_masks)
    for pair, match_indices in correspondences.items():
        indices_list = zip(*match_indices)
        for c, indices in zip(pair, indices_list):
            index_map = index_maps[c]
            kps = keypoints[c][0]
            for index in indices:
                if index not in index_map:
                    index_map[index] = len(index_map)
                    merged_keypoints[c].append(kps[index])
        c1, c2 = pair
        for i1, i2 in match_indices:
            merged_correspondences.setdefault(pair, []).append((index_maps[c1][i1], index_maps[c2][i2]))

In [21]:
sys.path.append(Root + 'OpenSfM/')
from opensfm import matching
features = { cam : mult_homo(np.linalg.inv(cameras[cam]), np.array([x.pt for x in kps])) for cam, kps in merged_keypoints.items() }
colors = { cam : [(0, 0, 0) for _ in kps] for cam, kps in merged_keypoints.items() }
run_config = { 'min_track_length' : 2, 'processes' : 5 }
tracks_graph = matching.create_tracks_graph(features, colors, merged_correspondences, run_config)

In [23]:
from opensfm.reconstruction import *
from opensfm import dataset
from opensfm import types
import opensfm.config
import yaml
reload(reconstruction)

class DataMock(dataset.DataSet):
    def __init__(self):
        self.reference_lla = None
        self.config = yaml.load(opensfm.config.default_config_yaml)
        self.config.update(run_config)
    
    def reference_lla_exists(self):
        return self.reference_lla is not None
    
    def save_reference_lla(self, ref):
        self.reference_lla = ref

    def load_reference_lla(self):
        return self.reference_lla
        
    def images(self):
        return list(range(1, Num_cameras + 1))
    
    def load_exif(self, cam_index):
        cam_mat = cameras[cam_index]
        return {
            'width' : Video_width / cam_mat[0, 0],
            'height' : Video_height / cam_mat[1, 1],
            'focal_prior' : 1,
            'camera' : cam_index
        }
    
    def load_tracks_graph(self):
        return tracks_graph
    
    def ground_control_points_exist(self):
        return False
    
    def load_camera_models(self):
        def create_camera(i):
            cam_mat = cameras[i]
            camera = types.PerspectiveCamera()
            camera.id = i
            camera.width = Video_width / cam_mat[0, 0]
            camera.height = Video_height / cam_mat[1, 1]
            camera.focal = camera.focal_prior = 1
            camera.k1 = camera.k1_prior = 0
            camera.k2 = camera.k2_prior = 0
            return camera
        return { i : create_camera(i) for i, camera in cameras.items() }
    
    def save_reconstruction(self, reconstructions, filename=None, minify=False):
        self.reconstructions = reconstructions
    
    def load_reconstruction(self, filename=None):
        return self.reconstructions
    
data = DataMock()

def incremental_reconstruction(data):
    """Run the entire incremental reconstruction pipeline."""
    logger.info("Starting incremental reconstruction")
    report = {}
    chrono = Chronometer()
    if not data.reference_lla_exists():
        data.invent_reference_lla()

    graph = data.load_tracks_graph()
    tracks, images = matching.tracks_and_images(graph)
    chrono.lap('load_tracks_graph')
    remaining_images = set(images)
    gcp = None
    if data.ground_control_points_exist():
        gcp = data.load_ground_control_points()
    common_tracks = matching.all_common_tracks(graph, tracks)
    reconstructions = []
    pairs = compute_image_pairs(common_tracks, data)
    chrono.lap('compute_image_pairs')
    report['num_candidate_image_pairs'] = len(pairs)
    report['reconstructions'] = []
    for im1, im2 in pairs:
        if im1 in remaining_images and im2 in remaining_images:
            rec_report = {}
            report['reconstructions'].append(rec_report)
            tracks, p1, p2 = common_tracks[im1, im2]
            reconstruction, rec_report['bootstrap'] = bootstrap_reconstruction(
                data, graph, im1, im2, p1, p2)

            if reconstruction:
                remaining_images.remove(im1)
                remaining_images.remove(im2)
                reconstruction, rec_report['grow'] = grow_reconstruction(
                    data, graph, reconstruction, remaining_images, gcp)
                reconstructions.append(reconstruction)
                reconstructions = sorted(reconstructions,
                                         key=lambda x: -len(x.shots))
                data.save_reconstruction(reconstructions)

    for k, r in enumerate(reconstructions):
        logger.info("Reconstruction {}: {} images, {} points".format(
            k, len(r.shots), len(r.points)))
    logger.info("{} partial reconstructions in total.".format(
        len(reconstructions)))
    chrono.lap('compute_reconstructions')
    report['wall_times'] = dict(chrono.lap_times())
    report['not_reconstructed_images'] = list(remaining_images)
    return report
# reconstruction.incremental_reconstruction = incremental_reconstruction
incremental_reconstruction(data)

{'not_reconstructed_images': [],
 'num_candidate_image_pairs': 12,
 'reconstructions': [{'bootstrap': {'common_tracks': 469,
    'decision': 'Success',
    'image_pair': (1, 2),
    'memory_usage': 4456210432,
    'triangulated_points': 391,
    'two_view_reconstruction': {'5_point_inliers': 391,
     'method': '5_point',
     'plane_based_inliers': 388}},
   'grow': {'steps': []}},
  {'bootstrap': {'common_tracks': 272,
    'decision': 'Success',
    'image_pair': (3, 4),
    'memory_usage': 4456210432,
    'triangulated_points': 248,
    'two_view_reconstruction': {'5_point_inliers': 246,
     'method': '5_point',
     'plane_based_inliers': 165}},
   'grow': {'steps': []}},
  {'bootstrap': {'common_tracks': 133,
    'decision': 'Success',
    'image_pair': (5, 6),
    'memory_usage': 4456210432,
    'triangulated_points': 111,
    'two_view_reconstruction': {'5_point_inliers': 111,
     'method': 'plane_based',
     'plane_based_inliers': 111}},
   'grow': {'steps': []}}],
 'wall_ti

In [436]:
rs = data.load_reconstruction()

In [437]:
transformations = { im : (x.pose.get_rotation_matrix(), x.pose.translation) for r in rs for im, x in r.shots.items() }
R1, t1 = transformations[1]
trans1 = {} # camera 1 point of reference
for im, (Ri, ti) in transformations.items():
    R = np.dot(Ri, R1.T)
    trans1[im] = (R, ti - np.dot(R, t1))
trans1

{1: (array([[  1.00000000e+00,   1.55548023e-18,  -6.93474255e-18],
         [  1.55548023e-18,   1.00000000e+00,  -2.94817128e-18],
         [ -6.93474255e-18,  -2.94817128e-18,   1.00000000e+00]]),
  array([ -4.44089210e-16,   0.00000000e+00,   0.00000000e+00])),
 2: (array([[ 0.07449526, -0.99720615, -0.00550825],
         [-0.13674328, -0.00474353, -0.99059516],
         [ 0.98780146,  0.07454786, -0.13671461]]),
  array([-0.36075196,  0.49436254,  4.07025784])),
 3: (array([[ 0.99988707,  0.01473158,  0.00297153],
         [-0.0146786 ,  0.99974561, -0.01712479],
         [-0.00322305,  0.01707923,  0.99984894]]),
  array([ 0.01138758,  0.45821072, -0.17173804])),
 4: (array([[ 0.04731907,  0.99880432, -0.01228126],
         [ 0.11376182,  0.0068263 ,  0.9934846 ],
         [ 0.99238055, -0.0484079 , -0.11330279]]),
  array([ 0.48957148, -0.43405104,  4.02746834])),
 5: (array([[  9.95930072e-01,   1.97614650e-02,  -8.79362095e-02],
         [ -1.98910448e-02,   9.99801975e-01,  -

In [438]:
Rts_test = [transformations[1], transformations[2]]

In [36]:
f_dict = frames[('soda_front', 800)]
fs = [f_dict[cam] for cam in Cam_ids]